In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
import glob
from PIL import Image

In [2]:
# Initialize the ImageDataGenerator class
datagen = ImageDataGenerator(
    rotation_range = 15,
    #shear_range = 0.2,
    zoom_range = 0.1,
    horizontal_flip = True,
    brightness_range = (0.5, 1.5)
)

(Option 1): Use images from a folder

In [4]:
# Generate the augmented images and save to the specified folder (save_to_dir)
# Assumes that the folder structure is a single folder that contains folders labelled for different emotions, each containing images. Examples:
    # BaseFolder\\Happy\\*.jpg
    # BaseFolder\\Surprise\\*.jpg

file_paths = glob.glob(f"Base Folder Name\\**\\*.jpg") 

for image in file_paths:
    emotion = image.split("\\")[1]
    im = load_img(image)
    x = img_to_array(im)
    x = x.reshape((1,) + x.shape)
    i = 0
    num_images = 20

    for batch in datagen.flow(x, batch_size=1, save_to_dir=f'Save To Folder Name/{emotion}/', save_prefix='aug_img', save_format='jpg'):
        i += 1
        if i >= num_images:
            break

(Option 2): Use images stored as pixels in a CSV file

In [ ]:
# Augmented images are stored as jpg files in the specified folder (save_to_dir), not as another CSV of pixels
expressions = {0:"Angry", 1:"Disgust", 2:"Fear", 3:"Happy", 4:"Sad", 5:"Surprise", 6:"Neutral"}
df = pd.read_csv("Images.csv")

for index, row in df.iterrows():
    emotion = row["emotion"]
    x = np.fromstring(row["pixels"], dtype=int, sep=" ").reshape(48, 48, 1).astype("float32")
    x = x.reshape((1,) + x.shape)
    i = 0
    num_images = 5

    for batch in datagen.flow(x, batch_size=1, save_to_dir=f'Save To Folder Name/{expressions[emotion]}/', save_prefix='aug_img', save_format='jpg'):
        i += 1
        if i >= num_images:
            break

Turn folder of images into an npy dataset

In [12]:
emotions = ["Happy", "Surprise", "Disgust", "Neutral", "Angry", "Fear", "Sad"]
expressions = {"Angry":0, "Disgust":1, "Fear":2, "Happy":3, "Sad":4, "Surprise":5, "Neutral":6}
l = []
shape=(224,224)

# turn folders of images into a dataframe
for emotion in emotions:
    images = glob.glob(f"Base Folder Name/{emotion}/*.jpg")
    for image in images:
        img = Image.open(image).convert('L')
        reshaped = np.array(img).reshape(shape[0]*shape[1])
        reshaped_as_string = map(str, reshaped)
        string = ' '.join(reshaped_as_string)
        l.append([expressions[emotion], string])

df = pd.DataFrame(l, columns=["emotion", "pixels"])

In [13]:
shape=(224,224)

datapoints = df['pixels'].tolist()

#getting features for training
X = []
for xseq in datapoints:
    xx = [int(xp) for xp in xseq.split(' ')]
    xx = np.asarray(xx).reshape(shape)
    X.append(xx.astype('float32'))

X = np.asarray(X)
X = np.expand_dims(X, -1)

#getting labels for training
y = pd.get_dummies(df['emotion']).values

np.save('fdataX', X)
np.save('flabels', y)

Combine npy datasets

In [15]:
dataX1 = np.load('file1_fdataX.npy')
dataY1 = np.load('file1_flabels.npy')

dataX2 = np.load('file2_fdataX.npy')
dataY2 = np.load('file2_flabels.npy')

In [16]:
x_result = np.concatenate([dataX1, dataX2])
y_result = np.concatenate([dataY1, dataY2])

In [12]:
np.save('fileName_fdataX', x_result)
np.save('fileName_flabels', y_result)